In [ ]:
# Binance API credentials
api_key = '0e94f8bfa93f5e4d0c3e64d4a8ab2bbf4ce891b80cc87f0f7baaf3aa7b34c445'
api_secret = '39ead8ebdd0ba4c88cd56d467f5274feaf82cd3994621309657b1978f096da16'

In [71]:
import os
import time
from binance.client import Client
from binance.exceptions import BinanceAPIException, BinanceOrderException

class GridBot:
    def __init__(self, symbol, api_key, api_secret, grid_spacing, quantity_per_trade, leverage):
        self.client = Client(api_key, api_secret, tld="futures", testnet=True)
        self.symbol = symbol
        self.grid_spacing = grid_spacing
        self.quantity_per_trade = quantity_per_trade
        self.leverage = leverage
        self.orders = []
        self.wins = 0
        self.losses = 0
        self.total_profit = 0.0

    def place_order(self, side, price, quantity):
        try:
            order = self.client.futures_create_order(
                symbol=self.symbol,
                side=side,
                type='LIMIT',  # Change this to 'MARKET' if you want market orders
                timeInForce='GTC',
                quantity=quantity,
                price=price,
                leverage=self.leverage
            )
            return order
        except BinanceAPIException as e:
            print(f"Error placing order: {e}")
            return None

    def buy(self, price):
        order = self.place_order("BUY", price, self.quantity_per_trade)
        if order:
            self.orders.append(order)

    def sell(self, price):
        order = self.place_order("SELL", price, self.quantity_per_trade)
        if order:
            self.orders.append(order)

    def grid_trade(self, lower_price, upper_price):
        current_price = float(self.client.futures_ticker(symbol=self.symbol)['lastPrice'])

        for price in range(int(lower_price), int(upper_price), self.grid_spacing):
            if price < current_price:
                self.buy(price)
            elif price > current_price:
                self.sell(price)

    def cancel_all_orders(self):
        for order in self.orders:
            try:
                self.client.futures_cancel_order(symbol=self.symbol, orderId=order['orderId'])
            except BinanceAPIException as e:
                print(f"Error canceling order: {e}")

        # Clear the orders list after canceling all orders
        self.orders = []

    def close_all_positions(self):
            positions = self.client.futures_position_information(symbol=self.symbol)
            print(positions)
            for position in positions:
                position_qty = float(position['positionAmt'])
                print(position_qty)
                if position_qty > 0:
                    self.sell(position_qty)
                elif position_qty < 0:
                    self.buy(abs(position_qty))
    def calculate_results(self):
        for order in self.orders:
            executed_price = float(order['executedQty']) * float(order['price'])
            if order['side'] == 'BUY':
                self.losses += executed_price
            else:
                self.wins += executed_price

        self.total_profit = self.wins - self.losses

    def display_results(self):
        print(f"Total Wins: {self.wins} USDT")
        print(f"Total Losses: {self.losses} USDT")
        print(f"Total Profit: {self.total_profit} USDT")

if __name__ == "__main__":
    # Set your Binance API key and secret
    api_key = '0e94f8bfa93f5e4d0c3e64d4a8ab2bbf4ce891b80cc87f0f7baaf3aa7b34c445'
    api_secret = '39ead8ebdd0ba4c88cd56d467f5274feaf82cd3994621309657b1978f096da16'

    # Set the trading parameters
    symbol = 'BTCUSDT'
    grid_spacing = 100  # Set your grid spacing in USDT
    quantity_per_trade = 0.01  # Adjusted the quantity per trade
    leverage = 10  # Set your desired leverage

    # Create an instance of the GridBot
    bot = GridBot(symbol, api_key, api_secret, grid_spacing, quantity_per_trade, leverage)

    # Define the lower and upper bounds of the grid
    lower_price = 38000
    upper_price = 48000

    # Run the bot for 24 hours
    duration_hours = 1
    end_time = time.time() + duration_hours * 3600

    while time.time() < end_time:
        bot.grid_trade(lower_price, upper_price)
        time.sleep(10)  # Adjust as needed to avoid rate limits

    # Cancel all open orders before exiting
    bot.close_all_positions()
    bot.cancel_all_orders()

    # Calculate and display results
    bot.calculate_results()
    bot.display_results()


Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error placing order: APIError(code=-2025): Reach max open order limit.
Error 

KeyboardInterrupt: ignored